In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [2]:
# load the model,scaler,label encoder and one hot encoder
model = load_model('modell.h5')
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder = pickle.load(f)
with open('onehot_encoder_geography.pkl', 'rb') as f:
    one_hot_encoder = pickle.load(f)

In [3]:
# example input date for features 	CreditScore	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	Geography_France	Geography_Germany	Geography_Spain
input_data = {
    'CreditScore': 600,
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000,
    'Geography':'France'
    
}

In [13]:
# one hot encoding for Geography
geo_encoded = one_hot_encoder.transform([[input_data['Geography']]]).toarray()
geo_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_df

c:\Users\USER\Documents\ML and NLP\Bank classification with ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
# combine the new one-hot encoded columns with the original dataframe
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df.drop(columns=['Geography']).reset_index(drop=True), geo_df], axis=1)
input_df

In [16]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,35,5,50000,2,1,1,60000,1.0,0.0,0.0


In [22]:
# encode Gender
input_df['Gender'] = label_encoder.transform([input_data['Gender']])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,35,5,50000,2,1,1,60000,1.0,0.0,0.0


In [23]:
# scaling the data
input_scaled = scaler.transform(input_df)

In [24]:
# prediction
prediction = model.predict(input_scaled)

1/1 [==============================] - 0s 174ms/step


In [25]:
prediction

array([[0.01539569]], dtype=float32)

In [28]:
prediction_proba = prediction[0][0]
if prediction_proba > 0.5:
    print(f"The customer is likely to leave the bank with a probability of {prediction_proba:.2f}")
else:
    print(f"The customer is likely to stay with the bank with a probability of {1 - prediction_proba:.2f}")

The customer is likely to stay with the bank with a probability of 0.98
